---
# Projeto 2 - Ciência dos Dados

Nome: Daniel Pucciariello

Nome: Stephanie Liu

A proposta desse trabalho é avaliar se um tweet sobre o tema "Pastel" foi feito por qual tipo de usuário: um usuário que gosta de Pastel e teve uma boa experiência decorrente do fato dele ter comido o alimento ou somente demonstra gostar desse, um usuário que desgoste de pastel ou teve uma má experiência ao se alimentar dele ou um usuário cujo emprego da palavra pastel tenha sido utilizada para expressar um sentimento que não se relacione com o alimento de forma direta (irrelevante).

---

___
# Classificador automático de sentimento
## Preparando o ambiente no jupyter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emoji
!pip install preprocessor

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle
import emoji
import preprocessor as p


In [4]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    
    text_subbed = re.sub(pattern, ' ', text)
    text_subbed=text_subbed.replace("\n", " ")
    text_subbed=text_subbed.replace("…", " ")
    text_subbed=text_subbed.replace("@", " ")
    text_subbed=text_subbed.replace("rt", "")
    text_subbed=text_subbed.replace("https", "")
    
    return text_subbed
#tirar coisas com https

In [5]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
# with open('auth.pass') as fp:    
#     data = json.load(fp)

#Configurando a biblioteca. Não modificar
# auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
# auth.set_access_token(data['access_token'], data['access_token_secret'])

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

---

In [6]:
data_treinamento = pd.read_excel("Excel_pastel.xlsx",sheet_name="Treinamento")

In [7]:
irrelevantes_raw = " ".join(data_treinamento[data_treinamento.Classe<=1].Treinamento)

In [8]:
irrelevantes = cleanup(irrelevantes_raw)
#irrelevantes

In [9]:
tabela_relativa_ngostam_irrelevantes = pd.Series(irrelevantes.lower().split()).value_counts(True)
tabela_absoluta_ngostam_irrelevantes = pd.Series(irrelevantes.lower().split()).value_counts(False)


In [10]:
gostam_raw = " ".join(data_treinamento[data_treinamento.Classe==2].Treinamento)
ngostam_raw = " ".join(data_treinamento[data_treinamento.Classe<2].Treinamento)

In [11]:
tabela_relativa_gostam = pd.Series(gostam_raw.lower().split()).value_counts(True)
tabela_absoluta_gostam = pd.Series(gostam_raw.lower().split()).value_counts(False)

In [12]:
Multiplicando_tabela_naogostam = tabela_relativa_ngostam_irrelevantes*2

In [13]:
tudo=irrelevantes_raw + gostam_raw
tudo=cleanup(tudo)
tudo_absoluto=pd.Series(tudo.lower().split()).value_counts(False)

In [14]:
data_teste = pd.read_excel("Excel_pastel.xlsx",sheet_name="Teste")

In [15]:
tudo_raw = tudo.split()
tudo_sem_repeticao = set(tudo_raw)

#tudo_sem_repeticao

In [16]:
#data_teste_idx= data_teste.set_index("Teste")
# vamos ter a nossa string relevante e string de não relevante
# vamos somar os dois = total
# vamos dar o split = lista total
# vamos jogaar a lista total no set --> conjunto de palavras unicas
# ao temos o len do set  -- numero das palavras unicas

# ao inves da freq relativa 

#     gosta=((tabela_absoluta_gostam[i.split()]+1)/(count_palavras_gostam+d)).prod()
#     ngosta=((tabela_absoluta_ngostam_irrelevantes[i.split()]+1)/(count_palavras_gostam+d)).prod()
lista=[]

#Variaveis gostam 
gostam_limpo = cleanup(gostam_raw)
palavras_gostam = gostam_limpo.split()
ngostam_limpo = cleanup(ngostam_raw)

#Variaveis que não gostam
palavras_ngostam = ngostam_limpo.split()
count_palavras_gostam = len(palavras_gostam)
count_palavras_ngostam = len(palavras_ngostam)

d = len(tudo_sem_repeticao)

#fazer if pra verificar se a palavra eh inedita ou nao; se for inedita, vai ser igual ao termo abaixo

n_existe = 1/(count_palavras_gostam+d)

#se der underflow, tem que fazer log 
for i in data_teste.Teste:
    gosta = 1
    ngosta = 1
    print(i.split())
    for x in i.split():
        prob_g = 0
        prob_n = 0
        if x not in tabela_absoluta_gostam:
            prob_g= 1/(count_palavras_gostam+d)
            
        elif x in tabela_absoluta_gostam:
            prob_g=((tabela_absoluta_gostam[x]+1)/(count_palavras_gostam+d))
            
        if x not in tabela_absoluta_ngostam_irrelevantes:
            prob_n= 1/(count_palavras_ngostam+d)
            
        elif x in tabela_absoluta_ngostam_irrelevantes:
            prob_n=((tabela_absoluta_ngostam_irrelevantes[x]+1)/(count_palavras_ngostam+d))
            
        gosta*=prob_g
        ngosta*=prob_n
    
    if gosta>ngosta:
        valor=2
    elif gosta<ngosta:
        valor=1
    a = tabela_absoluta_gostam[i.split()]+1
    lista.append(valor)
#print(valor_a)

data_teste.Classe = lista
data_teste.to_excel("Classificador_3_categ_v1.xlsx")


['"fazer', 'um', 'picadinho', 'de', 'rogério"', '"coxinha', 'coxinha"', '"fazer', 'pastel', 'e', 'vender', '1', 'por', '1,', '2', 'por', '2"', 'eu:', 'meninos,', 'limites']
['"o', 'tanto', 'de', 'pastel', 'de', 'flango', 'que', 'dá', 'pra', 'por', 'seloco"', 'https://t.co/ix7dyfo7cf']
['[10/09]', 'cínthia', '💙:', 'amiga', 'to', 'fazendo', 'pastel', 'de', 'brócolis', 'com', 'queijo', 'pra', 'ti', '🥳🥳🤩🤩', 'uma', 'amiga', 'dessas', '😍😍🤩🤩❤️❤️']
['@__guedes_', 'vamos', 'pra', 'praia,', 'shopping,', 'comer', 'pastel,', 'vem', 'pra', 'minha', 'casa...sla']
['@__mtcalma', 'vou', 'fritar', 'mais', 'depois', 'da', 'escola', 'kkkkkkk', 'queimou', 'tudo', 'esses', 'kkkkkkkkkkkkkkk', 'só', 'salvo', 'o', 'pastel']
['@_lovingdead', 'um', 'pastel', 'frio', 'e', 'um', 'suco', 'por', '6', 'reais..', 'sendo', 'que', 'o', 'pastel', 'novinho', '(nesse', 'mesmo', 'lugar', 'claro)', 'eh', '6', '(os', 'sabores', 'básicos', 'né)', 'e', '2,50', 'o', 'suco', 'kkkk', 'ótimo', 'negócio', 'economizar', '2,50', 'com

/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:851: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


['dia', 'icônico:', 'o', 'dia', 'q', 'a', 'macedo', 'cagou', 'a', 'manuela', 'a', 'pau', 'com', 'um', 'pastel', 'folhado', 'kkkkkkkkkkkkkkkkkkkkkkkkkkkk']
['diego', '"', 'vou', 'ir', 'pra', 'china', 'jogar', 'futebol"', 'eu', '"', 'vai', 'ir', 'é', 'para', 'conquistar', 'uma', 'chinesa', 'isso', 'sim', '"', 'aí', 'ele"', 'é', 'poh,vou', 'chegar', 'nela', 'oferecendo', 'pastel', 'de', '"flango"', '"', 'odeio', 'ele', 'pprt', 'kkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk']
['do', 'nada', 'minha', 'vo', 'chega', 'aqui', 'no', 'meu', 'trabalho', 'com', 'um', 'pastel', 'e', 'uma', 'empada', 'pra', 'mim,', 'ai', 'logo', 'em', 'seguida', 'entra', 'minha', 'mae', 'com', 'um', 'pedaço', 'de', 'torta', 'e', 'uma', 'fanta', 'uva', 'kkkkkk', 'ai', 'como', 'eu', 'amo']
['dois', 'likes', 'nesse', 'tweet', 'eu', 'compro', 'um', 'pastel', 'slipknot', 'https://t.co/hxlfi6m4qw']
['dps', 'de', 'comer', 'um', 'super', 'prato', 'ro', 'eu', 'aq', 'comendo', 'pastel']
['duas', 'roupas', 'que', 'eu', 'amo', 'usar', 'são

___
### Verificando a performance
Agora você deve testar o seu classificador com a base de Testes.

In [17]:
errou=len(data_teste.loc[(data_teste.Classe==2)&(data_teste.Modelo<=1)])
acuracia=1-(errou/(len(data_teste)))
acuracia

0.9053708439897699